# Original SVM Analysis

In [1]:
%load_ext autoreload
%autoreload 2
# to ensure kernel resets when files change around it
!pip install np ipympl
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls  # Check if required cuda 9.0 amd64-deb file is downloaded
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls /var/cuda-repo-9-0-local | grep .pub
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!sudo apt-get install cuda-9.0
!nvcc --version
!pip install thundersvm

--2020-12-14 04:43:19--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?4uZPaaygDSsfRc7HnAwiDa3akkfHSUIdydoeVvDB5sTKKfby5KYRxIEPu-E1pCmU1pr5SBaz4Vm4adYSsu_d178f4seIf199D1gP1pkxAQFKAyjzDiSeWmpLy7-1nfhhgCPOjgHDRUCgcUKdco15KhJOFk6QbjtO1KKWlZhwIRBgyKd51DD9wrNBYBuoPKqEckRzABP-ZztrPCVI36K2 [following]
--2020-12-14 04:43:19--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?4uZPaaygDSsfRc7HnAwiDa3akkfHSUIdydoeVvDB5sTKKfby5KYRxIEPu-E1pCmU1pr5SBaz4Vm4adYSsu_d178f4seIf199D1gP1pkxAQFKAyjzDiSe

In [2]:
import np # numerical analysis
import pandas as pd # main library for data analysis
import matplotlib.pyplot as plt # main library for data plotting

from IPython.display import display, Markdown

from sklearn.model_selection import train_test_split # Import train_test_split function
# Import svm model
from thundersvm import SVC
# Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import *

import warnings

"""
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)"""

warnings.filterwarnings('ignore') # literally to ignore all the warnings that don't matter

%matplotlib inline

def printf(*args, sep=" ", end="\n"):
    string = sep.join([str(i) for i in args])+end
    display(Markdown(string))

In [3]:
df = pd.read_csv("data.csv")
X = df[["x", "y", "z", "n", "freezeX", "freezeY", "freezeZ"]]
y = df.freeze
df

,x,y,z,n,freezeX,freezeY,freezeZ,freeze
0,-45.0,972.0,181.0,989.732287,0.0,0.0,0.0,0.0
1,-18.0,981.0,212.0,1003.807252,0.0,0.0,0.0,0.0
2,18.0,981.0,222.0,1005.966699,0.0,0.0,0.0,0.0
3,36.0,990.0,222.0,1015.224113,0.0,0.0,0.0,0.0
4,36.0,990.0,212.0,1013.084399,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
1140830,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1140831,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1140832,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1140833,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [4]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)
X_train

,x,y,z,n,freezeX,freezeY,freezeZ
980006,-145.0,1000.0,262.0,1043.872119,0.187473,0.777503,0.701144
397555,863.0,324.0,151.0,934.101708,0.000000,0.000000,0.000000
449215,-36.0,990.0,0.0,990.654329,0.000000,0.000000,0.000000
602553,327.0,953.0,40.0,1008.334270,4.233566,3.919225,1.858995
1034290,818.0,453.0,-191.0,954.365758,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
1115886,-109.0,37.0,-979.0,985.743882,0.000000,0.000000,0.000000
728621,172.0,1074.0,343.0,1140.486300,0.562288,0.705512,0.471921
664162,-172.0,944.0,252.0,992.080642,1.032315,0.509653,0.554239
625229,109.0,722.0,353.0,811.032675,0.689535,1.154806,1.192720


In [ ]:
kernel_args = [
    {"kernel": "rbf"},
    {"kernel": "linear"}
]

datasets = {
    "freezeXY": ["freezeX", "freezeY"],
    "freezeXZ": ["freezeX", "freezeZ"],
    "freezeYZ": ["freezeY", "freezeZ"],
    "freeze": ["freezeX", "freezeY", "freezeZ"]
}

preds = []

for kwargs in kernel_args:
    printf("# ", kwargs["kernel"])
    for name, data in datasets.items():
        printf("## ", name)
        
        #Create a svm Classifier
        svclassifier = SVC(**kwargs)
        
        # Train the model using the training sets
        svclassifier.fit(X_train[data], y_train)
        
        # Predict the response for test dataset
        y_pred = svclassifier.predict(X_test[data])
        preds.append(y_pred)
        
        # Model Accuracy: how often is the classifier correct?
        print(confusion_matrix(y_test,y_pred))
        print(classification_report(y_test,y_pred))
        
        # Classification Metrics
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("Balanced Accuracy:", balanced_accuracy_score(y_test, y_pred))
        print("Average Precison:", average_precision_score(y_test, y_pred))
        print("Neg Brier Score:", brier_score_loss(y_test, y_pred))
        print("F1:", f1_score(y_test, y_pred))
        print("-log loss:", log_loss(y_test, y_pred))
        print("Precision:", precision_score(y_test, y_pred))
        print("Recall:", recall_score(y_test, y_pred))
        print("Jaccard:", jaccard_score(y_test, y_pred))
        print("Roc Auc:", roc_auc_score(y_test, y_pred))
        print("\n\n\n")

#  rbf


##  freezeXY


[[306054   3141]
 [ 27351   5705]]
              precision    recall  f1-score   support

         0.0       0.92      0.99      0.95    309195
         1.0       0.64      0.17      0.27     33056

    accuracy                           0.91    342251
   macro avg       0.78      0.58      0.61    342251
weighted avg       0.89      0.91      0.89    342251

Accuracy: 0.9109074918700019
Balanced Accuracy: 0.5812136385291775
Average Precison: 0.19121987643776361
Neg Brier Score: 0.08909250812999817
F1: 0.27230203808887404
-log loss: nan
Precision: 0.64492425955234
Recall: 0.17258591481122942
Jaccard: 0.1576097466640882
Roc Auc: 0.5812136385291775






##  freezeXZ


[[306491   2704]
 [ 27510   5546]]
              precision    recall  f1-score   support

         0.0       0.92      0.99      0.95    309195
         1.0       0.67      0.17      0.27     33056

    accuracy                           0.91    342251
   macro avg       0.79      0.58      0.61    342251
weighted avg       0.89      0.91      0.89    342251

Accuracy: 0.9117197612278708
Balanced Accuracy: 0.579515302630553
Average Precison: 0.19316567912920285
Neg Brier Score: 0.08828023877212923
F1: 0.26853241659807287
-log loss: nan
Precision: 0.6722424242424242
Recall: 0.1677758954501452
Jaccard: 0.15508948545861298
Roc Auc: 0.579515302630553






...

##  freezeXZ


[[  2258 306937]
 [  3113  29943]]
              precision    recall  f1-score   support

         0.0       0.42      0.01      0.01    309195
         1.0       0.09      0.91      0.16     33056

    accuracy                           0.09    342251
   macro avg       0.25      0.46      0.09    342251
weighted avg       0.39      0.09      0.03    342251

Accuracy: 0.09408591939833631
Balanced Accuracy: 0.456564655531656
Average Precison: 0.08960849373531253
Neg Brier Score: 0.9059140806016637
F1: 0.16188205527442584
-log loss: nan
Precision: 0.08888328188078841
Recall: 0.9058264762826719
Jaccard: 0.08806946025359345
Roc Auc: 0.456564655531656






##  freezeYZ


[[308862    333]
 [ 32780    276]]
              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95    309195
         1.0       0.45      0.01      0.02     33056

    accuracy                           0.90    342251
   macro avg       0.68      0.50      0.48    342251
weighted avg       0.86      0.90      0.86    342251

Accuracy: 0.903249369614698
Balanced Accuracy: 0.5036362386606559
Average Precison: 0.09956165541015304
Neg Brier Score: 0.09675063038530202
F1: 0.016396851329273727
-log loss: nan
Precision: 0.45320197044334976
Recall: 0.00834946757018393
Jaccard: 0.0082661954535925
Roc Auc: 0.5036362386606559






##  freeze
